<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in Calgary</font></h1>


<img src = "Calgary1.jpg" width = 800> </a>


## Capstone Project - The Battle of Neighborhoods (Week 1)

#### 1. A description of the problem and a discussion of the background.

The energy capital, nature, mountains in addtion to the fourth highest population density makes the city of Calgary an attractive place for a buisness. However for newcomers, immigrants and even people from the Eastern provinces of Canada could be a big challnege to select the neighbourhood to set up the buiness. This study aims to answer most of the questions to select the best neighrbourhood in Calgary to set up a grocery store. comapre the neighbourhood considering the following factors to guide the investors to maximize their rate of return on investment specifically in during the current crisis time. The study is designed in particular for an investor who is interested to start up a Halal grocery store.

Following information to be used in order to make the buisness decission for the selection of neighbourhood:

- Accessibility of public transit
- Top 50 venues of each neighbourhood
- Population density of each neighbourhood
- Nearby famous landmark (Mall, Hospital, University)



#### 2. A description of the data and how it will be used to solve the problem

The Calgary nieghbourhood data and postal code will be scrapped from wikipedia using beautiful soup library. The population density of each neighbourhood will also be extracted from wikipedia and the two tables to be merged.
Now that we have built a dataframe of the postal code of each neighborhood along with the borough name, neighborhood name, population density, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

For latitute and longitude of metro stations, venues, Geocoder Python package will be utilized to get the geographical coordinates of a given postal code. In order to extract the latitude and logitude for all the neighbourhoods, a while loop will be employed in order to make sure that I get the coordinates for all of Calgary neighborhoods. Different machine learning algorithms will be applied including K-Means to determine the best neighbourhood for the start of buisness.


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.extra.rate_limiter import RateLimiter

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Calgary').text
soup=BeautifulSoup(url,'lxml')
table=soup.find('table')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Name<sup class="reference" id="cite_ref-2012censusreport_9-1"><a href="#cite_note-2012censusreport-9">[9]</a></sup>
</th>
<th align="center">Quadrant
</th>
<th>Sector<sup class="reference" id="cite_ref-opendatacommunities_10-0"><a href="#cite_note-opendatacommunities-10">[10]</a></sup>
</th>
<th align="center">Ward<sup class="reference" id="cite_ref-2013censusreport_11-0"><a href="#cite_note-2013censusreport-11">[11]</a></sup>
</th>
<th>Type<sup class="reference" id="cite_ref-opendatacommunities_10-1"><a href="#cite_note-opendatacommunities-10">[10]</a></sup>
</th>
<th align="center">2012 <br/>Population<br/>Rank
</th>
<th align="center">Population<br/>(2012)<sup class="reference" id="cite_ref-2012censusreport_9-2"><a href="#cite_note-2012censusreport-9">[9]</a></sup>
</th>
<th align="center">Population<br/>(2011)<sup class="reference" id="cite_ref-2012censusreport_9-3"><a href="#cite_note-2012censusreport-9">[9]</a></sup>
</th>
<th al

#### Extract the table from html

In [4]:
output_rows = []

for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)

Calgary=pd.DataFrame(output_rows)
Calgary.columns=['Name','Quadrant','Sector', 'Ward','Type','2012 Population Rank','Population (2012)','Population (2011)','% change','Dwellings (2012)','Area (km2)','Population density']
# Calgary

print('Orignial', Calgary.shape[0])

tas=Calgary[Calgary.Type != 'Industrial']

print('\nWihtout industrial', tas.shape[0])

# removing the first and last row
tag=tas.drop(tas.index[0],inplace=True)
tag=tas.drop(tas.index[-1],inplace=True)

tag=tas.reset_index(drop=True)

print('\nWihtout industrial and first and last row ', tag.shape[0])

tag ['Population density']=tag['Population density'].str.replace('\n','')
tag

Orignial 259

Wihtout industrial 217

Wihtout industrial and first and last row  215


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Name,Quadrant,Sector,Ward,Type,2012 Population Rank,Population (2012),Population (2011),% change,Dwellings (2012),Area (km2),Population density
0,Abbeydale,NE/SE,Northeast,10,Residential,82,"5,917","5,700",3.8,"2,023",1.7,"3,480.6"
1,Acadia,SE,South,9,Residential,27,"10,705","10,615",0.8,"5,053",3.9,"2,744.9"
2,Albert Park/Radisson Heights,SE,East,10,Residential,75,"6,234","6,217",0.3,"2,709",2.5,"2,493.6"
3,Altadore,SW,Centre,11,Residential,39,"9,116","8,907",2.3,"4,486",2.9,"3,143.4"
4,Applewood Park,SE/NE,East,10,Residential,69,"6,498","6,404",1.5,"2,215",1.6,"4,061.3"
5,Arbour Lake,NW,Northwest,2,Residential,26,"10,836","10,762",0.7,"3,918",4.4,"2,462.7"
6,Aspen Woods,SW,West,6,Residential,92,"5,271","4,469",17.9,"2,281",3.8,"1,387.1"
7,Auburn Bay,SE,Southeast,12,Residential,60,"7,193","5,769",24.7,"2,808",4.5,"1,598.4"
8,Banff Trail,NW,Centre,7,Residential,113,"3,837","3,582",7.1,"1,950",1.5,"2,558"
9,Bankview,SW,Centre,8,Residential,94,"5,221","4,754",9.8,"3,528",0.7,"7,458.6"


Drop columns and cleaning the data

In [5]:
# clean up the dataset to remove unnecessary columns (eg. Ward) 
tag.drop(tag.columns[[3,4,5,7,8,9]], axis=1, inplace=True)  # df.columns is zero-based pd.Index 

# df=df[df.Sector!= '']
print(tag.shape[0])
tag.head()

215


,Name,Quadrant,Sector,Population (2012),Area (km2),Population density
0,Abbeydale,NE/SE,Northeast,"5,917",1.7,"3,480.6"
1,Acadia,SE,South,"10,705",3.9,"2,744.9"
2,Albert Park/Radisson Heights,SE,East,"6,234",2.5,"2,493.6"
3,Altadore,SW,Centre,"9,116",2.9,"3,143.4"
4,Applewood Park,SE/NE,East,"6,498",1.6,"4,061.3"


In [6]:
#Dropping rows with Residual since they dont have populaiton density

df=tag[~tag.Name.str.contains("Residual")]
print(df.shape)

df = df.rename(columns={'Population density': 'Population_density'})
df = df.rename(columns={'Population (2012)': 'Population_2012'})
df = df.rename(columns={'Area (km2)': 'Area_km2'})

# for sake of consistency, let's also make all column labels of type string
df.columns = list(map(str, df.columns))

# # set the name as index - useful for quickly looking up neighbourhood using .loc method
# df.set_index('Name', inplace = True)

df.dtypes

(201, 6)


Name                  object
Quadrant              object
Sector                object
Population_2012       object
Area_km2              object
Population_density    object
dtype: object

In [7]:
# Delete rows with no population density data

df.dropna(subset=['Population_density'], inplace=True)
print (df.shape)
df

(201, 6)


,Name,Quadrant,Sector,Population_2012,Area_km2,Population_density
0,Abbeydale,NE/SE,Northeast,"5,917",1.7,"3,480.6"
1,Acadia,SE,South,"10,705",3.9,"2,744.9"
2,Albert Park/Radisson Heights,SE,East,"6,234",2.5,"2,493.6"
3,Altadore,SW,Centre,"9,116",2.9,"3,143.4"
4,Applewood Park,SE/NE,East,"6,498",1.6,"4,061.3"
5,Arbour Lake,NW,Northwest,"10,836",4.4,"2,462.7"
6,Aspen Woods,SW,West,"5,271",3.8,"1,387.1"
7,Auburn Bay,SE,Southeast,"7,193",4.5,"1,598.4"
8,Banff Trail,NW,Centre,"3,837",1.5,"2,558"
9,Bankview,SW,Centre,"5,221",0.7,"7,458.6"


In [8]:
df.apply(pd.to_numeric, errors='ignore')
df.dtypes

Name                  object
Quadrant              object
Sector                object
Population_2012       object
Area_km2              object
Population_density    object
dtype: object

In [9]:
# convert into float to pandas array and then reassign to dataframe
df['Population_density']=df['Population_density'].str.replace(',','')
df['Population_2012']=df['Population_2012'].str.replace(',','')
df['Area_km2'] = pd.to_numeric(df['Area_km2'])
df['Population_2012'] = pd.to_numeric(df['Population_2012'])
df['Population_density'] = pd.to_numeric(df['Population_density'])

df.dtypes

Name                   object
Quadrant               object
Sector                 object
Population_2012       float64
Area_km2              float64
Population_density    float64
dtype: object

In [10]:
#Dropping rows with 0 and no population density
df.dropna(subset=['Population_density'], inplace=True)
df=df.sort_values(by=['Population_density'], ascending=False).reset_index(drop=True)
df

,Name,Quadrant,Sector,Population_2012,Area_km2,Population_density
0,Lower Mount Royal,SW,Centre,3180.0,0.3,10600.0
1,Mission,SW/SE,Centre,4325.0,0.5,8650.0
2,Chinatown,SW/SE,Centre,1577.0,0.2,7885.0
3,Bankview,SW,Centre,5221.0,0.7,7458.6
4,Beltline,SW/SE,Centre,19681.0,2.9,6786.6
5,Downtown Commercial Core,SW/SE,Centre,8015.0,1.3,6165.4
6,Taradale,NE,Northeast,16841.0,2.9,5807.2
7,Downtown West End,SW,Centre,2322.0,0.4,5805.0
8,Downtown East Village,SE,Centre,2782.0,0.5,5564.0
9,Castleridge,NE,Northeast,6096.0,1.2,5080.0


In [11]:
n=8
df.drop(df.tail(n).index,inplace=True) # drop last n rows

df


,Name,Quadrant,Sector,Population_2012,Area_km2,Population_density
0,Lower Mount Royal,SW,Centre,3180.0,0.3,10600.0
1,Mission,SW/SE,Centre,4325.0,0.5,8650.0
2,Chinatown,SW/SE,Centre,1577.0,0.2,7885.0
3,Bankview,SW,Centre,5221.0,0.7,7458.6
4,Beltline,SW/SE,Centre,19681.0,2.9,6786.6
5,Downtown Commercial Core,SW/SE,Centre,8015.0,1.3,6165.4
6,Taradale,NE,Northeast,16841.0,2.9,5807.2
7,Downtown West End,SW,Centre,2322.0,0.4,5805.0
8,Downtown East Village,SE,Centre,2782.0,0.5,5564.0
9,Castleridge,NE,Northeast,6096.0,1.2,5080.0


shape the dataframe

In [12]:
df.shape

(191, 6)

### Get the latitue and logitude of the Calgary City

In [13]:
address = 'Calgary, AB'

geolocator = Nominatim(user_agent="AB_explorer")
location = geolocator.geocode(address)
latitude_C = location.latitude
longitude_C = location.longitude
print('The geograpical coordinate of Calgary are {}, {}.'.format(latitude_C, longitude_C))

The geograpical coordinate of Calgary are 51.0534234, -114.0625892.


In [14]:
# neigh =df['Name']

# latitude=[]
# longitude=[]

# geolocator = Nominatim(user_agent="AB_explorer")

# for neighbourhood in df.Name:  
#     try:
#         input = neighbourhood
#         location = geolocator.geocode(input, timeout=15)
#         latitude.append(location.latitude)
#         longitude.append(location.longitude)
# #     except:
# #         print('Error, skipping address...', e)

# # print('The geograpical coordinate of Calgary are {}, {}.'.format(latitude, longitude))
# # df_geocodes = pd.DataFrame('coordinate':latitude)

# # print(latitude)

In [15]:
df['Name'] = df.Name.map(str) + " ,Calgary"
locator = Nominatim(user_agent="AB_explorer")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# 2- - create location column
df['location'] = df['Name'].apply(geocode)

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
# create a new column called POINT and fill column with coordinates as tuple pulled from the Location column

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.dtypes

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Erin Woods ,Calgary',), **{}).
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 956, in send
    self.connect()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1392, in connect
    server_hostname=server_hostname)


Name                   object
Quadrant               object
Sector                 object
Population_2012       float64
Area_km2              float64
Population_density    float64
location               object
point                  object
latitude              float64
longitude             float64
altitude              float64
dtype: object

In [16]:
df.dtypes

Name                   object
Quadrant               object
Sector                 object
Population_2012       float64
Area_km2              float64
Population_density    float64
location               object
point                  object
latitude              float64
longitude             float64
altitude              float64
dtype: object

In [17]:
# df.location[0]
df['location'] = df['location'].astype('str')
df['Postal_code']=df['location'].str.split(",").str[3].tolist()
df

,Name,Quadrant,Sector,Population_2012,Area_km2,Population_density,location,point,latitude,longitude,altitude,Postal_code
0,"Lower Mount Royal ,Calgary",SW,Centre,3180.0,0.3,10600.0,"Lower Mount Royal, Calgary, Alberta, T2T 0K5, ...","(51.036645, -114.0871388, 0.0)",51.036645,-114.087139,0.0,T2T 0K5
1,"Mission ,Calgary",SW/SE,Centre,4325.0,0.5,8650.0,"Mission, Calgary, Alberta, T2S 1P5, Canada","(51.0317576, -114.0667199, 0.0)",51.031758,-114.066720,0.0,T2S 1P5
2,"Chinatown ,Calgary",SW/SE,Centre,1577.0,0.2,7885.0,"Chinatown, Calgary, Alberta, T2G 2B6, Canada","(51.0506542, -114.0626107, 0.0)",51.050654,-114.062611,0.0,T2G 2B6
3,"Bankview ,Calgary",SW,Centre,5221.0,0.7,7458.6,"Bankview, Calgary, Alberta, T2T 4M8, Canada","(51.0338869, -114.0995178, 0.0)",51.033887,-114.099518,0.0,T2T 4M8
4,"Beltline ,Calgary",SW/SE,Centre,19681.0,2.9,6786.6,"Beltline, Calgary, Alberta, T2R 0K4, Canada","(51.0404978, -114.0725934, 0.0)",51.040498,-114.072593,0.0,T2R 0K4
5,"Downtown Commercial Core ,Calgary",SW/SE,Centre,8015.0,1.3,6165.4,"Downtown Commercial Core, Calgary, Alberta, T2...","(51.0473777, -114.0671989, 0.0)",51.047378,-114.067199,0.0,T2P 0R2
6,"Taradale ,Calgary",NE,Northeast,16841.0,2.9,5807.2,"Taradale, Calgary, Alberta, T3J 3E6, Canada","(51.1167042, -113.9384638, 0.0)",51.116704,-113.938464,0.0,T3J 3E6
7,"Downtown West End ,Calgary",SW,Centre,2322.0,0.4,5805.0,"Downtown West End, Calgary, Alberta, T2P 2B3, ...","(51.0475537, -114.0834198, 0.0)",51.047554,-114.083420,0.0,T2P 2B3
8,"Downtown East Village ,Calgary",SE,Centre,2782.0,0.5,5564.0,"Downtown East Village, Calgary, Alberta, T2G 1...","(51.0464963, -114.0506433, 0.0)",51.046496,-114.050643,0.0,T2G 1K7
9,"Castleridge ,Calgary",NE,Northeast,6096.0,1.2,5080.0,"Castleridge, Calgary, Alberta, T3J 1P6, Canada","(51.1059774, -113.9598199, 0.0)",51.105977,-113.959820,0.0,T3J 1P6


In [18]:
df = df.drop(['point', 'altitude'], axis=1)
df

,Name,Quadrant,Sector,Population_2012,Area_km2,Population_density,location,latitude,longitude,Postal_code
0,"Lower Mount Royal ,Calgary",SW,Centre,3180.0,0.3,10600.0,"Lower Mount Royal, Calgary, Alberta, T2T 0K5, ...",51.036645,-114.087139,T2T 0K5
1,"Mission ,Calgary",SW/SE,Centre,4325.0,0.5,8650.0,"Mission, Calgary, Alberta, T2S 1P5, Canada",51.031758,-114.066720,T2S 1P5
2,"Chinatown ,Calgary",SW/SE,Centre,1577.0,0.2,7885.0,"Chinatown, Calgary, Alberta, T2G 2B6, Canada",51.050654,-114.062611,T2G 2B6
3,"Bankview ,Calgary",SW,Centre,5221.0,0.7,7458.6,"Bankview, Calgary, Alberta, T2T 4M8, Canada",51.033887,-114.099518,T2T 4M8
4,"Beltline ,Calgary",SW/SE,Centre,19681.0,2.9,6786.6,"Beltline, Calgary, Alberta, T2R 0K4, Canada",51.040498,-114.072593,T2R 0K4
5,"Downtown Commercial Core ,Calgary",SW/SE,Centre,8015.0,1.3,6165.4,"Downtown Commercial Core, Calgary, Alberta, T2...",51.047378,-114.067199,T2P 0R2
6,"Taradale ,Calgary",NE,Northeast,16841.0,2.9,5807.2,"Taradale, Calgary, Alberta, T3J 3E6, Canada",51.116704,-113.938464,T3J 3E6
7,"Downtown West End ,Calgary",SW,Centre,2322.0,0.4,5805.0,"Downtown West End, Calgary, Alberta, T2P 2B3, ...",51.047554,-114.083420,T2P 2B3
8,"Downtown East Village ,Calgary",SE,Centre,2782.0,0.5,5564.0,"Downtown East Village, Calgary, Alberta, T2G 1...",51.046496,-114.050643,T2G 1K7
9,"Castleridge ,Calgary",NE,Northeast,6096.0,1.2,5080.0,"Castleridge, Calgary, Alberta, T3J 1P6, Canada",51.105977,-113.959820,T3J 1P6


In [19]:
df.shape

(191, 10)

In [20]:
# NUmber of neighbourhoods in each quadrant
df['Quadrant'].value_counts()

SW       67
NW       48
SE       36
NE       22
SW/SE     9
NW/NE     4
NE/NW     2
NE/SE     2
SE/NE     1
Name: Quadrant, dtype: int64

In [21]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Name'].unique()),
        df.shape[0]
    )
)

The dataframe has 191 boroughs and 191 neighborhoods.


In [22]:
df.dropna(subset=['latitude'], inplace=True)
print (df.shape)


(187, 10)


#### Create a map of Calgary with neighborhoods superimposed on top.

In [23]:
# create map of Calgary using latitude and longitude values
map_calgary = folium.Map(location=[latitude_C, longitude_C], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['latitude'], df['longitude'], df['Name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  
    
map_calgary

#### Define Foursquare Credentials and Version

In [24]:
CLIENT_ID = 'KU5GGGC5WIKUQVE33J1YPQHR0CYOK0BTBGGFBZBNQH5IWRV4' # your Foursquare ID
CLIENT_SECRET = 'VXSZVINDIO3BGR4MJ1QDOE5M1MGHLZD2XN5HP4SRMCLWFJUT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KU5GGGC5WIKUQVE33J1YPQHR0CYOK0BTBGGFBZBNQH5IWRV4
CLIENT_SECRET:VXSZVINDIO3BGR4MJ1QDOE5M1MGHLZD2XN5HP4SRMCLWFJUT


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [65]:
df.loc[0, 'Name']

'Lower Mount Royal ,Calgary'

#### Now, let's get the top 30 venues that are in Lower Mount Royal, Calgary  within a radius of 500 meters.

Get the neighborhood's latitude and longitude values.

In [66]:
neighborhood_latitude = df.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lower Mount Royal ,Calgary are 51.036645, -114.0871388.


In [67]:
search_query = 'Grocery'
radius = 500
print(search_query + ' .... OK!')

Grocery .... OK!


In [68]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KU5GGGC5WIKUQVE33J1YPQHR0CYOK0BTBGGFBZBNQH5IWRV4&client_secret=VXSZVINDIO3BGR4MJ1QDOE5M1MGHLZD2XN5HP4SRMCLWFJUT&ll=51.036645,-114.0871388&v=20180605&query=Grocery&radius=500&limit=30'

In [69]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb50a4daba297001c333900'},
 'response': {'venues': [{'id': '4b0586eff964a520cd7622e3',
    'name': 'Kalamata Grocery',
    'location': {'address': '1421 11th St. SW',
     'lat': 51.03940743942703,
     'lng': -114.08912946314987,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.03940743942703,
       'lng': -114.08912946314987}],
     'distance': 337,
     'postalCode': 'T2R 1G7',
     'cc': 'CA',
     'city': 'Calgary',
     'state': 'AB',
     'country': 'Canada',
     'formattedAddress': ['1421 11th St. SW', 'Calgary AB T2R 1G7', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588923367',
    'hasPerk': False}]}}

In [30]:
results.keys()

dict_keys(['meta', 'response'])

In [70]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b0586eff964a520cd7622e3,1421 11th St. SW,CA,Calgary,Canada,337,"[1421 11th St. SW, Calgary AB T2R 1G7, Canada]","[{'label': 'display', 'lat': 51.03940743942703...",51.039407,-114.089129,T2R 1G7,AB,Kalamata Grocery,v-1588923367


In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Kalamata Grocery,Grocery Store,1421 11th St. SW,CA,Calgary,Canada,337,"[1421 11th St. SW, Calgary AB T2R 1G7, Canada]","[{'label': 'display', 'lat': 51.03940743942703...",51.039407,-114.089129,T2R 1G7,AB,4b0586eff964a520cd7622e3


In [33]:
dataframe_filtered.name

0    Kalamata Grocery
Name: name, dtype: object

Now we are ready to clean the json and structure it into a *pandas* dataframe.

#### Let's create a function to repeat the same process to all the neighborhoods in Calgary

In [ ]:
results['response']['venues'][0]['id']

In [90]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
      
        # create the API request URL       
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            lat,
            lng,
            VERSION,
            search_query,
            radius, LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['venues'][0]['id']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [91]:
Calgary_venues = getNearbyVenues(names=df['Name'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Lower Mount Royal ,Calgary


TypeError: string indices must be integers

#### Let's check the size of the resulting dataframe

In [ ]:
print(Calgary_venues.shape)
Calgary_venues.head()

Let's check how many venues were returned for each neighborhood

In [ ]:
Calgary_venues.groupby('Neighborhood').count()

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

Calgary_grouped_clustering = Calgary_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Calgary_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 